___

<a href='https://github.com/eliasmelul/'> <img src='https://s3.us-east-2.amazonaws.com/wordontheamazon.com/NoMargin_NewLogo.png' style='width: 15em;' align='right' /></a>
# Finding my Schitt's Creek
#### Data Collection: Venue Data
___
<h3 align="right">by Elias Melul, Data Scientist </h3> 

___



## Collecting Venue Data using Foursquare's API

---
**GOOD NEWS!** To use the recommender system, you don't need to rerun and scrap the web. I've made the data available for you **<a href='https://github.com/eliasmelul/finding_schitts/blob/master/Data/one_hot_encoded_usa_FS.csv'>here</a>**.

Want direct access? **<a href='https://s3.us-east-2.amazonaws.com/www.findingmyschittscreek.com/Data/one_hot_encoded_usa_FS.csv'>Click Here</a>**

---
This data is a one-hot encoded dataframe of what category of venues are in the top 100 in every city.

----


In [1]:
import pandas as pd # import pandas for dataframes
import numpy as np
import requests
from bs4 import BeautifulSoup
import locale
from datetime import datetime
import re
import matplotlib.pyplot as plt
import seaborn as sns
import json
import folium
from pandas.io.json import json_normalize


import matplotlib.cm as cm
import matplotlib.colors as colors

from IPython.display import HTML, display
from IPython.display import Image 
from IPython.core.display import HTML 

pd.options.display.max_columns = None
pd.options.display.max_rows=None


%matplotlib inline

In [2]:
scraped_data = pd.read_csv('https://s3.us-east-2.amazonaws.com/www.findingmyschittscreek.com/Data/scraped_datausa.csv', index_col=0)
scraped_data.head()

,City,Population,Population Change,Poverty Rate,Median Age,Median Household Income,Median Household Income Change,Number Employees,Number Employees Change,Median Property Value,Median Property Value Change,Average Male Salary,Average Female Salary,Gender Salary Ratio M2F,Gini 2018,Gini Change,Patient to Clinician Ratio,Foreign Born Population Ratio,Citizens Percentage,Total Degrees,Degrees Ratio M2F,Degrees per Capita,Households,People Per House,Homeownership,Commute Time
0,"Alabaster, AL",32567.0,0.923,11.1,37.0,74383.0,1.44,15674.0,-0.716,167800.0,0.902,58549.0,40615.0,1.441561,0.46,0.436681,1166.0,6.130,94.9,3.0,0.000000,0.000092,10800.0,3.015463,82.7,28.7
1,"Alexander City, AL",40756.0,-0.493,21.2,43.3,42181.0,7.07,15317.0,1.450,107000.0,1.710,58549.0,40615.0,1.441561,0.46,0.436681,1535.0,1.350,99.2,350.0,0.612903,0.008588,16400.0,2.485122,71.9,24.5
2,"Aliceville, AL",2466.0,-23.400,34.6,36.3,24097.0,20.90,810.0,-18.600,102800.0,5.440,58549.0,40615.0,1.441561,0.46,0.436681,2981.0,0.243,99.8,0.0,1.000000,NaN,928.0,2.657328,48.4,21.1
3,"Andalusia, AL",8918.0,-1.370,17.3,41.1,36101.0,6.14,3629.0,7.460,110400.0,12.800,58549.0,40615.0,1.441561,0.46,0.436681,1991.0,1.180,99.7,514.0,0.903704,0.057636,3610.0,2.470360,61.3,15.8
4,"Ashland, AL",2113.0,-5.290,24.6,45.7,27264.0,-12.20,868.0,1.640,133000.0,48.400,58549.0,40615.0,1.441561,0.46,0.436681,3389.0,2.980,97.0,0.0,1.000000,NaN,895.0,2.360894,50.8,24.1


**Collecting and Adding location data**

To use Foursquare's API, we must first add a latitude and longitude attribute to all the cities possible.

In [ ]:
# list_cities = list(scraped_data.City)
# list_cities = [i+", USA" for i in list_cities]

# from geopy.geocoders import Nominatim
# geolocator = Nominatim(user_agent="new_fkf_finding_my_shitts_creek")
# locations_list = []

# for i in list_cities:
#     try:
#         location = geolocator.geocode(i, timeout=10)
#         city = i[:-5]
#         lat = location.latitude
#         long = location.longitude
#         dic_lats = {"City":city,"Latitude":lat,"Longitude":long}
#         locations_list.append(dic_lats)
#         print(dic_lats)
#     except:
#         try:
#             geolocator = Nominatim(user_agent="wowzadayum_lol")
#             location = geolocator.geocode(i, timeout=10)
#             city = i[:-5]
#             lat = location.latitude
#             long = location.longitude
#             dic_lats = {"City":city,"Latitude":lat,"Longitude":long}
#             locations_list.append(dic_lats)
#         except:
#             try:
#                 geolocator = Nominatim(user_agent="hello_its_me_FMSC")
#                 location = geolocator.geocode(i, timeout=10)
#                 city = i[:-5]
#                 lat = location.latitude
#                 long = location.longitude
#                 dic_lats = {"City":city,"Latitude":lat,"Longitude":long}
#                 locations_list.append(dic_lats)
#             except:
#                 pass


I first tried getting the data using Geopy. Unfortunately, it was too volatile and unreliable and had to look for other solutions.

I found <a href='https://simplemaps.com/data/us-cities'>this</a> website from which you can download the data. 

In [5]:
cities_latlong = pd.read_csv('https://s3.us-east-2.amazonaws.com/www.findingmyschittscreek.com/Data/uscities.csv')
cities_latlong.head()

,city,city_ascii,state_id,state_name,county_fips,county_name,county_fips_all,county_name_all,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
0,South Creek,South Creek,WA,Washington,53053,Pierce,53053,Pierce,46.9994,-122.3921,2500.0,125.0,polygon,False,True,America/Los_Angeles,3,98580 98387 98338,1840042075
1,Roslyn,Roslyn,WA,Washington,53037,Kittitas,53037,Kittitas,47.2507,-121.0989,947.0,84.0,polygon,False,True,America/Los_Angeles,3,98941 98068 98925,1840019842
2,Sprague,Sprague,WA,Washington,53043,Lincoln,53043,Lincoln,47.3048,-117.9713,441.0,163.0,polygon,False,True,America/Los_Angeles,3,99032,1840021107
3,Gig Harbor,Gig Harbor,WA,Washington,53053,Pierce,53053,Pierce,47.3352,-122.5968,9507.0,622.0,polygon,False,True,America/Los_Angeles,3,98332 98335,1840019855
4,Lake Cassidy,Lake Cassidy,WA,Washington,53061,Snohomish,53061,Snohomish,48.0639,-122.0920,3591.0,131.0,polygon,False,True,America/Los_Angeles,3,98223 98258 98270,1840041959


To use this dataset and begin scraping venue information for each city, all we need to do is create a city variable that includes the state, and combine the general data and the location dataset.

In [6]:
cities_latlong['City'] = [row.city+", "+row.state_id for i, row in cities_latlong.iterrows()]
cities_latlong.City[0:5]

0     South Creek, WA
1          Roslyn, WA
2         Sprague, WA
3      Gig Harbor, WA
4    Lake Cassidy, WA
Name: City, dtype: object

In [8]:
cities_loc_all = scraped_data.merge(cities_latlong, how='inner', on="City")
print(cities_loc_all.shape)
cities_loc_all.head()

(3958, 45)


,City,Population,Population Change,Poverty Rate,Median Age,Median Household Income,Median Household Income Change,Number Employees,Number Employees Change,Median Property Value,Median Property Value Change,Average Male Salary,Average Female Salary,Gender Salary Ratio M2F,Gini 2018,Gini Change,Patient to Clinician Ratio,Foreign Born Population Ratio,Citizens Percentage,Total Degrees,Degrees Ratio M2F,Degrees per Capita,Households,People Per House,Homeownership,Commute Time,city,city_ascii,state_id,state_name,county_fips,county_name,county_fips_all,county_name_all,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
0,"Alabaster, AL",32567.0,0.923,11.1,37.0,74383.0,1.44,15674.0,-0.716,167800.0,0.902,58549.0,40615.0,1.441561,0.46,0.436681,1166.0,6.130,94.9,3.0,0.000000,0.000092,10800.0,3.015463,82.7,28.7,Alabaster,Alabaster,AL,Alabama,1117,Shelby,01117,Shelby,33.2198,-86.8226,33202.0,507.0,polygon,False,True,America/Chicago,3,35114 35007 35144,1840013756
1,"Alexander City, AL",40756.0,-0.493,21.2,43.3,42181.0,7.07,15317.0,1.450,107000.0,1.710,58549.0,40615.0,1.441561,0.46,0.436681,1535.0,1.350,99.2,350.0,0.612903,0.008588,16400.0,2.485122,71.9,24.5,Alexander City,Alexander City,AL,Alabama,1123,Tallapoosa,01123,Tallapoosa,32.9242,-85.9361,9222.0,134.0,polygon,False,True,America/Chicago,3,35010 35011,1840013788
2,"Aliceville, AL",2466.0,-23.400,34.6,36.3,24097.0,20.90,810.0,-18.600,102800.0,5.440,58549.0,40615.0,1.441561,0.46,0.436681,2981.0,0.243,99.8,0.0,1.000000,NaN,928.0,2.657328,48.4,21.1,Aliceville,Aliceville,AL,Alabama,1107,Pickens,01107,Pickens,33.1237,-88.1594,2335.0,197.0,polygon,False,True,America/Chicago,3,35442,1840013758
3,"Andalusia, AL",8918.0,-1.370,17.3,41.1,36101.0,6.14,3629.0,7.460,110400.0,12.800,58549.0,40615.0,1.441561,0.46,0.436681,1991.0,1.180,99.7,514.0,0.903704,0.057636,3610.0,2.470360,61.3,15.8,Andalusia,Andalusia,AL,Alabama,1039,Covington,01039,Covington,31.3101,-86.4781,6486.0,172.0,polygon,False,True,America/Chicago,3,36420 36421,1840004591
4,"Ashland, AL",2113.0,-5.290,24.6,45.7,27264.0,-12.20,868.0,1.640,133000.0,48.400,58549.0,40615.0,1.441561,0.46,0.436681,3389.0,2.980,97.0,0.0,1.000000,NaN,895.0,2.360894,50.8,24.1,Ashland,Ashland,AL,Alabama,1027,Clay,01027,Clay,33.2704,-85.8337,1932.0,101.0,polygon,False,True,America/Chicago,3,36251,1840006738


Awesome! We may have lost a few cities in merging datasets, but it's acceptable for our overall analysis. With about 4000 cities and their profiles, the analysis will be quite comprehensible.

---

Now that we have the cities and their locations, lets begin using the Foursquare API.

---

**Exploring Venue Information from Foursquare**

In [9]:
CLIENT_ID = '###' # Your Foursquare ID
CLIENT_SECRET = '###' # Your Foursquare Secret
VERSION = '20180604'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1GFAHOQNHDJNYCFLI3EDIE1UHGZIJPVRTQIIRYVZGUXV1F23
CLIENT_SECRET:KYZMRBJB11ZS1YJITM4M2X0NH3THZVHTV5OWKFCOY1EVTDQM


In [15]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [18]:
LIMIT=100
def get_Nearby_Venues(names, latitudes, longitudes, radius=5000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
                    
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
try: 
    usa_venues = get_Nearby_Venues(names=cities_loc_all.City, latitudes=cities_loc_all.lat, longitudes=cities_loc_all.lng)
except:
    try:
        usa_venues = get_Nearby_Venues(names=cities_loc_all.City, latitudes=cities_loc_all.lat, longitudes=cities_loc_all.lng)
    except:
        usa_venues = get_Nearby_Venues(names=cities_loc_all.City, latitudes=cities_loc_all.lat, longitudes=cities_loc_all.lng)
print(usa_venues.shape)
usa_venues.head()

(146014, 7)


,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Alabaster, AL",33.2198,-86.8226,Alabaster YMCA,33.218549,-86.827118,Gym
1,"Alabaster, AL",33.2198,-86.8226,Starbucks,33.214867,-86.825894,Coffee Shop
2,"Alabaster, AL",33.2198,-86.8226,Publix,33.212939,-86.826002,Grocery Store
3,"Alabaster, AL",33.2198,-86.8226,Champy's Famous Fried Chicken,33.235264,-86.822617,Fried Chicken Joint
4,"Alabaster, AL",33.2198,-86.8226,Chick-fil-A,33.229871,-86.803642,Fast Food Restaurant


Now, we have a dataframe with all the information windowed over for every venue. Lets make it into a binary dataframe.

In [20]:
usa_venues.groupby('City').count()[0:10]

,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
City,,,,,,
"Abbeville, GA",4,4,4,4,4,4
"Aberdeen, ID",2,2,2,2,2,2
"Aberdeen, MD",99,99,99,99,99,99
"Aberdeen, SD",90,90,90,90,90,90
"Aberdeen, WA",61,61,61,61,61,61
"Abilene, KS",34,34,34,34,34,34
"Abilene, TX",100,100,100,100,100,100
"Abingdon, VA",76,76,76,76,76,76
"Ada, MN",4,4,4,4,4,4


In [21]:
print('There are {} uniques categories.'.format(len(usa_venues['Venue Category'].unique())))

There are 611 uniques categories.


In [22]:
#Get the dummy variables for each venue category
usa_dummies = pd.get_dummies(usa_venues[['Venue Category']], prefix="", prefix_sep="")

#Add name of city to dataframe
usa_dummies['City'] = usa_venues['City']

#Lets make it readable by changing the location of the City column to be the first column of the dataframe
move_column = [usa_dummies.columns[-1]] + list(usa_dummies.columns[:-1])
usa_dummies = usa_dummies[move_column]

print(usa_dummies.shape)
usa_dummies.head()

(146014, 611)


,Zoo Exhibit,ATM,Accessories Store,Acupuncturist,Adult Boutique,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,Airport Tram,Alternative Healer,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Apres Ski Bar,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Assisted Living,Astrologer,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bathing Area,Bavarian Restaurant,Bay,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boarding House,Boat Rental,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Bowling Green,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Bulgarian Restaurant,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cable Car,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Capitol Building,Car Wash,Caribbean Restaurant,Carpet Store,Casino,Castle,Cave,Cemetery,Cheese Shop,Child Care Service,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Cidery,City,City Hall,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Administrative Building,College Arts Building,College Auditorium,College Baseball Diamond,College Basketball Court,College Bookstore,College Cafeteria,College Communications Building,College Football Field,College Gym,College Hockey Rink,College Library,College Quad,College Rec Center,College Residence Hall,College Soccer Field,College Stadium,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Community College,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Country Dance Club,Coworking Space,Credit Union,Creperie,Cricket Ground,Cruise,Cuban Restaurant,Cupcake Shop,Curling Ice,Cycle Studio,Dance Studio,Daycare,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distillery,Distribution Center,Dive Bar,Dive Shop,Dive Spot,Doctor's Office,Dog Run,Donburi Restaurant,Doner Restaurant,Donut Shop,Drive-in Theater,Drugstore,Dry Cleaner,Dumpling Restaurant,Dutch Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Elementary School,Emergency Room,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Fabric Shop,Factory,Fair,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Festival,Field,Filipino Restaurant,Film Studio,Financial or Legal Service,Fire Station,Fireworks Store,Fish & Chips Shop,Fish Market,Fishing Spot,Fishing Store,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Hill,Himalayan Restaurant,Historic Site,H

In [24]:
usa_venues_grouped = usa_dummies.groupby('City').sum().reset_index()
print(usa_venues_grouped.shape)
usa_venues_grouped.head()

(3928, 611)


,City,Zoo Exhibit,ATM,Accessories Store,Acupuncturist,Adult Boutique,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,Airport Tram,Alternative Healer,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Apres Ski Bar,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Assisted Living,Astrologer,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bathing Area,Bavarian Restaurant,Bay,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boarding House,Boat Rental,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Bowling Green,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Bulgarian Restaurant,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cable Car,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Capitol Building,Car Wash,Caribbean Restaurant,Carpet Store,Casino,Castle,Cave,Cemetery,Cheese Shop,Child Care Service,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Cidery,City Hall,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Administrative Building,College Arts Building,College Auditorium,College Baseball Diamond,College Basketball Court,College Bookstore,College Cafeteria,College Communications Building,College Football Field,College Gym,College Hockey Rink,College Library,College Quad,College Rec Center,College Residence Hall,College Soccer Field,College Stadium,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Community College,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Country Dance Club,Coworking Space,Credit Union,Creperie,Cricket Ground,Cruise,Cuban Restaurant,Cupcake Shop,Curling Ice,Cycle Studio,Dance Studio,Daycare,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distillery,Distribution Center,Dive Bar,Dive Shop,Dive Spot,Doctor's Office,Dog Run,Donburi Restaurant,Doner Restaurant,Donut Shop,Drive-in Theater,Drugstore,Dry Cleaner,Dumpling Restaurant,Dutch Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Elementary School,Emergency Room,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Fabric Shop,Factory,Fair,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Festival,Field,Filipino Restaurant,Film Studio,Financial or Legal Service,Fire Station,Fireworks Store,Fish & Chips Shop,Fish Market,Fishing Spot,Fishing Store,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Forest,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Hill,Himalayan Restaurant,Historic Site,H

In [25]:
#usa_venues_grouped.to_csv("C:/...")

Great! Now we have the frequency of each category of venue of the top 100 venues in every city.